In [2]:
import torch
print(torch.cuda.is_available())         # should return True
print(torch.cuda.get_device_name(0))     # should say "NVIDIA GeForce RTX 3060"


True
NVIDIA GeForce RTX 3060


Imports 

In [5]:
import os

# For data manipulation and analysis 
import pandas as pd

# For numerical operations and array handling 
import numpy as np

# For creating plots and visualisations
import matplotlib.pylab as plt

# For advanced data visualisations 
#import seaborn as sns
# For file pattern matching 
import glob as glob

# For audio processing 
import librosa

# For displaying audio data visually 
import librosa.display

# For playing audio directly in notebooks 
import IPython.display as ipd

import wave

#for reproduability 
import random

#for the fodler re-structure 150 sample 
import shutil





Loading dataset 

In [6]:
base_dir = os.path.join(os.getcwd(), 'datasets', 'voice-based-id-recognition')
diffPhrase = os.path.join(base_dir,'differentPhrase')
samePhrase = os.path.join(base_dir,'samePhrase')

Checking Dataset Location existance

In [7]:
# Check if the training directory exists
if os.path.exists(diffPhrase):
    print("differentPhrase Directory: ")
    print(os.listdir(diffPhrase))
else:
    print("Directory does not exist:", diffPhrase) 
   

# Check if the testing directory exists
if os.path.exists(samePhrase):
    
    print("samePhrase Directory: ")
    print(os.listdir(samePhrase))
else:
    print("Directory does not exist:", samePhrase)

differentPhrase Directory: 
['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
samePhrase Directory: 
['1', 

Listen to audio files

In [8]:
"""

file_path = r"C:\Users\Aadil\Desktop\Voice-Auth-DF-System\ml-models\datasets\voice-based-id-recognition\differentPhrase\1\1-11.wav"

with wave.open(file_path, 'rb') as wav_file:
    num_frames = wav_file.getnframes()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    channels = wav_file.getnchannels()
    audio_data = wav_file.readframes(num_frames)

# Convert raw bytes to numpy array
audio_np = np.frombuffer(audio_data, dtype=np.int16)

# If stereo, reshape
if channels == 2:
    audio_np = audio_np.reshape(-1, 2)

ipd.Audio(audio_np, rate=sample_rate)


"""


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 18-19: truncated \UXXXXXXXX escape (3963867248.py, line 22)

Pre-Processing 

In [ ]:
#Dataset split
base_path = os.path.join(os.getcwd(), 'datasets', 'vox1_dev')
print(base_path)

#Each folder contains a different speaker

speaker_count = sum(os.path.isdir(os.path.join(base_path, entry)) for entry in os.listdir(base_path))
print("Number of original Speakers : ",speaker_count)
#Select a random selection of speakers, as inital dataset of 1211 speakers is roughly 30GB. will take a sample of 150 speakers(12.3%), aroudn 3.7GB

""""
# Destination path for subset
train_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')
os.makedirs(train_path, exist_ok=True)


# Set seed and select random 150 speakers
random.seed(42)
all_speakers = [entry for entry in os.listdir(base_path)
                if os.path.isdir(os.path.join(base_path, entry))]
selected_speakers = random.sample(all_speakers, 150)

# Copy selected speakers and their full contents to the subset folder
for speaker in selected_speakers:
    src_speaker_path = os.path.join(base_path, speaker)
    dest_speaker_path = os.path.join(train_path, speaker)
    shutil.copytree(src_speaker_path, dest_speaker_path)

#print(f"Subset dataset created at: {train_path}")
"""

# Print the selected speaker folders
count = 0
#print("Selected speakers:")
for speaker in selected_speakers:
    count +=1
    #print(speaker)

print("Number of train Speakers : ",count)



test_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_test')

test_speaker_count = sum(os.path.isdir(os.path.join(test_path, entry)) for entry in os.listdir(test_path))
print("Number of test Speakers : ",test_speaker_count)

testing to see if files are copied in both train and test, tested using hash of the files to compare. 

In [ ]:
import os
import hashlib

def file_hash(filepath):
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as f:
        hasher.update(f.read())
    return hasher.hexdigest()

# paths
test_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_test')
train_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

# Map filenames to full paths
def get_file_map(base_path):
    file_map = {}
    for root, _, files in os.walk(base_path):
        for file in files:
            file_map[file] = os.path.join(root, file)
    return file_map

test_files = get_file_map(test_path)
train_files = get_file_map(train_path)

# Compare common files
common = set(test_files).intersection(train_files)
print(f"Found {len(common)} common files.\n")

for filename in sorted(common):
    test_file = test_files[filename]
    train_file = train_files[filename]

    same = file_hash(test_file) == file_hash(train_file)
    print(f"{filename} - {'Same' if same else 'Different'}")



Spliting train and validaion(80/20 split)

In [ ]:

# Path to the dataset with 150 speaker folders
source_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

# Output subfolders for train and val
train_folder = os.path.join(source_path, 'train_data')
val_folder = os.path.join(source_path, 'val_data')

# Create output folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Get all speaker folders 
speaker_folders = [folder for folder in os.listdir(source_path)
                   if os.path.isdir(os.path.join(source_path, folder)) and folder not in ['train_data', 'val_data']]

# Shuffle and split 80/20
random.seed(42)
random.shuffle(speaker_folders)
split_index = int(0.8 * len(speaker_folders))
train_speakers = speaker_folders[:split_index]
val_speakers = speaker_folders[split_index:]

# Copy folders to train_data
for speaker in train_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(train_folder, speaker)
    shutil.move(src, dst)

# Copy folders to val_data
for speaker in val_speakers:
    src = os.path.join(source_path, speaker)
    dst = os.path.join(val_folder, speaker)
    shutil.move(src, dst)

print(f"{len(train_speakers)} speakers moved to train_data, {len(val_speakers)} to val_data.")



checking number of files, i.e the size to see how long it may take

In [9]:
import os

# Root dataset path
source_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150')

# Train and validation folders
train_folder = os.path.join(source_path, 'train_data')
val_folder = os.path.join(source_path, 'val_data')

def count_wav_files(folder_path):
    wav_count = 0
    for root, _, files in os.walk(folder_path):
        wav_files = [f for f in files if f.lower().endswith('.wav')]
        wav_count += len(wav_files)
    return wav_count

train_count = count_wav_files(train_folder)
val_count = count_wav_files(val_folder)
total_count = train_count + val_count

print(f"Train .wav files: {train_count}")
print(f"Val .wav files: {val_count}")
print(f"Total .wav files: {total_count}")


#find avreage audio file length to see what the set time shodul be for the pre proicessing
import librosa
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
def check_audio_durations(folder_path, sr=22050, plot_hist=True):
    folder = Path(folder_path)
    durations = []

    print(f"Scanning audio files in {folder}...")

    for wav_file in folder.rglob("*.wav"):
        try:
            y, _ = librosa.load(wav_file, sr=sr)
            duration = librosa.get_duration(y=y, sr=sr)
            durations.append(duration)
        except Exception as e:
             print(f"Failed to load {wav_file}: {e}")

    durations = np.array(durations)

    print(f"\n Duration Stats (in seconds):")
    print(f"Average: {np.mean(durations):.2f}s")
    print(f"Min:     {np.min(durations):.2f}s")
    print(f"Max:     {np.max(durations):.2f}s")
    print(f"Std dev: {np.std(durations):.2f}s")
    print(f"Total files: {len(durations)}")

    if plot_hist:
        plt.hist(durations, bins=50, edgecolor='black')
        plt.title("Audio Duration Distribution")
        plt.xlabel("Duration (seconds)")
        plt.ylabel("Number of files")
        plt.grid(True)
        plt.show()

    return durations

check_audio_durations('datasets/vox1_subset_150/train_data')
check_audio_durations('datasets/vox1_subset_150/val_data')
check_audio_durations('datasets/vox1_subset_test')


Train .wav files: 14308
Val .wav files: 3353
Total .wav files: 17661
Scanning audio files in datasets\vox1_subset_150\train_data...


Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x000001FB4716A680>
Traceback (most recent call last):
  File "c:\Users\Aadil\anaconda3\envs\voice-auth-env\lib\site-packages\llvmlite\binding\executionengine.py", line 178, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


Failed to load datasets\vox1_subset_150\train_data\id10002\0_laIeN-Q44\00001.wav: no compiled object yet for <Library '<numba.np.ufunc.wrappers._GufuncWrapper object at 0x000001FB4CA5F1F0>' at 0x1fb4ca39e10>


: 

Data Pre processing 

In [ ]:
import librosa
import noisereduce as nr
import numpy as np
import soundfile as sf
from pathlib import Path

def preprocess_audio(y, sr, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.6, normalise='rms'):
    
    # Trim leading/trailing silence
    y, _ = librosa.effects.trim(y)

    # Apply noise reduction / dereverberation
    if apply_reduction:
        y = nr.reduce_noise(y=y, sr=sr)

    # Apply pre-emphasis filter
    if apply_preemphasis:
        y = librosa.effects.preemphasis(y, coef=coef)

    # Normalisation method
    if normalise == 'rms':
        rms = np.sqrt(np.mean(y**2))
        y = y / (rms + 1e-6)
    elif normalise == 'peak':
        y = y / (np.max(np.abs(y)) + 1e-6)

    # Duration control: pad or truncate
    target_length = int(sr * target_duration)
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)))
    else:
        y = y[:target_length]

    return y

def preprocess_folder(input_folder, output_folder, sr=22050, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.6, normalise='rms'):
    
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    wav_files = list(input_folder.rglob("*.wav"))
    
    print(f"Found {len(wav_files)} audio files in {input_folder}")

    for wav_file in wav_files:
        try:
            y, _ = librosa.load(wav_file, sr=sr)
            y = preprocess_audio(
                y, sr, 
                target_duration=target_duration,
                apply_preemphasis=apply_preemphasis,
                apply_reduction=apply_reduction,
                coef=coef,
                normalise=normalise
            )

            # Save processed file to the same structure
            relative_path = wav_file.relative_to(input_folder)
            out_path = output_folder / relative_path
            out_path.parent.mkdir(parents=True, exist_ok=True)

            sf.write(out_path, y, sr)

        except Exception as e:
            print(f"Failed: {wav_file} → {e}")





In [ ]:


preprocess_folder(
    input_folder='datasets/vox1_subset_150/train_data',
    output_folder='datasets/vox1_subset_150/train_data_preprocessed',
    sr=22050,
    target_duration=3.0,
    apply_preemphasis=True,
    apply_reduction=True,
    coef=0.6,
    normalise='rms'
)


preprocess_folder(
    input_folder='datasets/vox1_subset_150/val_data',
    output_folder='datasets/vox1_subset_150/val_data_preprocessed',
    sr=22050,
    target_duration=3.0,
    apply_preemphasis=True,
    apply_reduction=True,
    coef=0.6,
    normalise='rms'
)


preprocess_folder(
    input_folder='datasets/vox1_subset_test',
    output_folder='datasets/vox1_subset_test_data_preprocessed',
    sr=22050,
    target_duration=3.0,
    apply_preemphasis=True,
    apply_reduction=True,
    coef=0.6,
    normalise='rms'
)


In [ ]:

import librosa
import noisereduce as nr
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

def preview_preprocessing(file_path, sr=22050, target_duration=6.0, apply_preemphasis=False, apply_reduction=False, coef=0.6, normalise='rms'):

    y, _ = librosa.load(file_path, sr=sr)
    y_orig = y.copy()

    # Trim silence
    y, _ = librosa.effects.trim(y)

    if apply_reduction:
        y = nr.reduce_noise(y=y, sr=sr)

    if apply_preemphasis:
        y = librosa.effects.preemphasis(y, coef=coef)

    # Choose normalisation method
    if normalise == 'rms':
        rms = np.sqrt(np.mean(y**2))
        y = y / (rms + 1e-6)
    elif normalise == 'peak':
        y = y / (np.max(np.abs(y)) + 1e-6)

    # Fix length
    target_len = int(sr * target_duration)
    if len(y) < target_len:
        y = np.pad(y, (0, target_len - len(y)))
    else:
        y = y[:target_len]

    print("Original:")
    ipd.display(ipd.Audio(y_orig, rate=sr))

    print("Preprocessed:")
    ipd.display(ipd.Audio(y, rate=sr))

    # Plot waveform comparison
    plt.figure(figsize=(12, 4))
    plt.plot(y_orig, label="Original", alpha=0.6)
    plt.plot(y, label="Preprocessed", alpha=0.6)
    plt.title("Waveform Comparison")
    plt.legend()
    plt.tight_layout()
    plt.show()


preview_preprocessing('datasets/vox1_subset_150/train_data/id10115/4cQ5Zgr-BK0/00001.wav', apply_preemphasis=True, apply_reduction=True, normalise='rms')





Feature extraction

In [ ]:
#need to extact MFCC, Chroma, Tonnetz, Mel spectrogram, spectral contrast, all can be done using librosa

#first split the data itno 5 parts for each model, as each model will have one fearture

import os
import shutil
import random
from pathlib import Path


def split_train_data_by_feature(train_data_path, output_base_path):
    speaker_dirs = [d for d in os.listdir(train_data_path)
                    if os.path.isdir(os.path.join(train_data_path, d))]

    #  consistent randomisation with ealier seed number for reducablity 
    random.seed(42)
    random.shuffle(speaker_dirs)

    feature_folders = [
        'denseNN_mfcc',
        'denseNN_chroma',
        'denseNN_tonnetz',
        'denseNN_spectral_contrast',
        'cnn_mel_spectrogram'
    ]

    part_size = len(speaker_dirs) // len(feature_folders)

    for i, feature_folder in enumerate(feature_folders):
        part_dir = os.path.join(output_base_path, feature_folder)
        os.makedirs(part_dir, exist_ok=True)

        start = i * part_size
        end = (i + 1) * part_size if i < len(feature_folders) - 1 else len(speaker_dirs)
        part_speakers = speaker_dirs[start:end]

        for speaker in part_speakers:
            src = os.path.join(train_data_path, speaker)
            dst = os.path.join(part_dir, speaker)
            shutil.copytree(src, dst)

        print(f"{feature_folder}: {len(part_speakers)} speakers copied.")


train_data_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150', 'train_data')
output_base_path = os.path.join(os.getcwd(), 'datasets', 'vox1_subset_150/vox1_split_by_feature')


#ACTUAL FEATURE EXTRACTING

def extract_feature(y, sr, feature_type):
    if feature_type == "mfcc":
        return librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    elif feature_type == "chroma":
        stft = np.abs(librosa.stft(y))
        return librosa.feature.chroma_stft(S=stft, sr=sr)
    elif feature_type == "tonnetz":
        y_harmonic = librosa.effects.harmonic(y)
        return librosa.feature.tonnetz(y=y_harmonic, sr=sr)
    elif feature_type == "spectral_contrast":
        stft = np.abs(librosa.stft(y))
        return librosa.feature.spectral_contrast(S=stft, sr=sr)
    elif feature_type == "mel_spectrogram":
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        return librosa.power_to_db(mel_spec, ref=np.max)
    else:
        raise ValueError(f"Unknown feature type: {feature_type}")

def extract_features_from_folder(input_dir, feature_type, output_base="datasets/vox1_subset_150/features", sr=22050):
    input_dir = Path(input_dir)
    input_name = input_dir.name
    output_dir = Path(output_base) / input_name
    output_dir.mkdir(parents=True, exist_ok=True)

    for wav_path in input_dir.rglob("*.wav"):
        try:
            y, _ = librosa.load(wav_path, sr=sr)
            feat = extract_feature(y, sr, feature_type)

            # Build output path, preserving structure
            relative = wav_path.relative_to(input_dir).with_suffix(".npy")
            out_path = output_dir / relative
            out_path.parent.mkdir(parents=True, exist_ok=True)

            np.save(out_path, feat)

        except Exception as e:
            print(f"Failed: {wav_path} → {e}")
    
    print(f"Done extracting '{feature_type}' features from {input_dir}")




extract_features_from_folder("datasets/vox1_subset_150/vox1_split_by_feature/denseNN_mfcc", "mfcc")
extract_features_from_folder("datasets/vox1_subset_150/vox1_split_by_feature/denseNN_tonnetz", "tonnetz")
extract_features_from_folder("datasets/vox1_subset_150/vox1_split_by_feature/denseNN_chroma", "chroma")
extract_features_from_folder("datasets/vox1_subset_150/vox1_split_by_feature/denseNN_spectral_contrast", "spectral_contrast")
extract_features_from_folder("datasets/vox1_subset_150/vox1_split_by_feature/cnn_mel_spectrogram", "mel_spectrogram")




Analysing some features 

Model creation 





In [ ]:
#CNN model

import torch
import torch.nn as nn
import torch.nn.functional as F





[]
